### Google Sheet Processing

This script transforms the "Clinical Benchmark and LLM" Sheet into a sheet that is more friendly to process.

This was used to create CoT-Difference-Sheet and CoT-Difference-Sheet-Invalid.

____

In [1]:
import pandas as pd
import json
import os
import sys

def load_sheet(sheet_path):
    '''
    Input: path to excel sheet
    Output: pandas dataframe of excel sheet
    '''
    sheet = pd.ExcelFile(sheet_path)

    return sheet

def define_columns():
    '''
    Function that defines all the columns to be used in final DataFrame

    Input: None
    Output: Dictionary mapping Column_Name --> Empty List
    '''

    columns = {
        "Model Name": [],
        "Model Size": [],
        "Model Domain": [],
        "Task Name": [],
        "Task Type": [],
        "Direct Score": [],
        "CoT Score": [],
        "Direct Invalid Score": [],
        "CoT Invalid Score": [], 
        "Difference": [],
        "Relative Difference": [],
        "Invalid Difference": [],
        "Invalid Relative Difference": []
    }

    return columns

def define_models():
    '''
    Returns a list of all models used in the sheet
    '''
    models = [
        "Baichuan-M1-14B-Instruct",
        "DeepSeek-R1",
        "DeepSeek-R1-Distill-Llama-8B",
        "DeepSeek-R1-Distill-Llama-70B",
        "DeepSeek-R1-Distill-Qwen-1.5B",
        "DeepSeek-R1-Distill-Qwen-7B",
        "DeepSeek-R1-Distill-Qwen-14B",
        "DeepSeek-R1-Distill-Qwen-32B",
        "gemma-2-9b-it",
        "gemma-2-27b-it",
        "gemma-3-1b-it",
        "gemma-3-4b-it",
        "gemma-3-12b-it",
        "gemma-3-27b-it",
        "Llama-3.1-8B-Instruct",
        "Llama-3.1-70B-Instruct",
        "Llama-3.2-1B-Instruct",
        "Llama-3.2-3B-Instruct",
        "Llama-3.3-70B-Instruct",
        "Llama-4-Scout-17B-16E-Instruct",
        "Llama-3.1-Nemotron-70B-Instruct-HF",
        "meditron-7b",
        "meditron-70b",
        "MeLLaMA-13B-chat",
        "MeLLaMA-70B-chat",
        "Llama3-OpenBioLLM-8B",
        "Llama3-OpenBioLLM-70B",
        "MMed-Llama-3-8B",
        "Llama-3.1-8B-UltraMedical",
        "Llama-3-70B-UltraMedical",
        "Ministral-8B-Instruct-2410",
        "Mistral-Small-Instruct-2409",
        "Mistral-Small-24B-Instruct-2501",
        "Mistral-Small-3.1-24B-Instruct-2503",
        "Mistral-Large-Instruct-2411",
        "BioMistral-7B",
        "Phi-3.5-mini-instruct",
        "Phi-3.5-MoE-instruct",
        "Phi-4",
        "Qwen2.5-1.5B-Instruct",
        "Qwen2.5-3B-Instruct",
        "Qwen2.5-7B-Instruct",
        "Qwen2.5-72B-Instruct",
        "QwQ-32B-Preview",
        "QWQ-32B",
        "Athene-V2-Chat",
        "Yi-1.5-9B-Chat-16K",
        "Yi-1.5-34B-Chat-16K",
        "gpt-35-turbo-0125",
        "gpt-4o-0806",
        "gemini-2.0-flash-001",
        "gemini-1.5-pro-002"
    ]

    return models


model_mapping = {
    "Baichuan-M1-14B-Instruct": [14, "med"],
    "DeepSeek-R1": [671, "gen"],
    "DeepSeek-R1-Distill-Llama-8B": [8, "gen"],
    "DeepSeek-R1-Distill-Llama-70B": [70, "gen"],
    "DeepSeek-R1-Distill-Qwen-1.5B": [1.5, "gen"],
    "DeepSeek-R1-Distill-Qwen-7B": [7, "gen"],
    "DeepSeek-R1-Distill-Qwen-14B": [14, "gen"],
    "DeepSeek-R1-Distill-Qwen-32B": [32, "gen"],
    "gemma-2-9b-it": [9, "gen"],
    "gemma-2-27b-it": [27, "gen"],
    "gemma-3-1b-it": [1, "gen"],
    "gemma-3-4b-it": [4, "gen"],
    "gemma-3-12b-it": [12, "gen"],
    "gemma-3-27b-it": [27, "gen"],
    "Llama-3.1-8B-Instruct": [8, "gen"],
    "Llama-3.1-70B-Instruct": [70, "gen"],
    "Llama-3.2-1B-Instruct": [1, "gen"],
    "Llama-3.2-3B-Instruct": [3, "gen"],
    "Llama-3.3-70B-Instruct": [70, "gen"],
    "Llama-4-Scout-17B-16E-Instruct": [109, "gen"],
    "Llama-3.1-Nemotron-70B-Instruct-HF": [70, "gen"],
    "meditron-7b": [7, "med"],
    "meditron-70b": [70, "med"],
    "MeLLaMA-13B-chat": [13, "med"],
    "MeLLaMA-70B-chat": [70, "med"],
    "Llama3-OpenBioLLM-8B": [8, "med"],
    "Llama3-OpenBioLLM-70B": [70, "med"],
    "MMed-Llama-3-8B": [8, "med"],
    "Llama-3.1-8B-UltraMedical": [8, "med"],
    "Llama-3-70B-UltraMedical": [70, "med"],
    "Ministral-8B-Instruct-2410": [8, "gen"],
    "Mistral-Small-Instruct-2409": [22, "gen"],
    "Mistral-Small-24B-Instruct-2501": [24, "gen"],
    "Mistral-Small-3.1-24B-Instruct-2503": [24, "gen"],
    "Mistral-Large-Instruct-2411": [123, "gen"],
    "BioMistral-7B": [7, "med"],
    "Phi-3.5-mini-instruct": [4, "gen"],
    "Phi-3.5-MoE-instruct": [42, "gen"],
    "Phi-4": [14, "gen"],
    "Qwen2.5-1.5B-Instruct": [1.5, "gen"],
    "Qwen2.5-3B-Instruct": [3, "gen"],
    "Qwen2.5-7B-Instruct": [7, "gen"],
    "Qwen2.5-72B-Instruct": [72, "gen"],
    "QwQ-32B-Preview": [32, "gen"],
    "QWQ-32B": [32, "gen"],
    "Athene-V2-Chat": [72, "gen"],
    "Yi-1.5-9B-Chat-16K": [9, "gen"],
    "Yi-1.5-34B-Chat-16K": [34, "gen"],
    "gpt-35-turbo-0125": [None, "gen"],
    "gpt-4o-0806": [None, "gen"],
    "gemini-2.0-flash-001": [None, "gen"],
    "gemini-1.5-pro-002": [None, "gen"],
}

def create_csv(data, output_path):
    '''
    Input: 
        - data: dictionary of information for the CSV
        - output_path: string path to output csv

    Output:
        - None: saves CSV directly to your path

    '''
    df = pd.DataFrame(data)

    df.to_csv(output_path)
    print(f"CSV successfully created at {output_path}")
    return

def filter_score(str_score):
    '''
    Input:
        - str_score: string representation of score and range

    Output:
        - score: rounded float of score
    '''

    str_score = str_score.split(" ")[0]
    score = float(str_score)

    return score

columns = define_columns()

all_models = define_models()


In [2]:
def process_sheet(google_sheet, sheet_names, all_models, task_type):
    '''
    Inputs:
        - google_sheet: pandas dataframe of the google sheet
        - sheet_name: names of the specific sheet to process in the format [Direct Sheet, CoT Sheet]

    Outputs:
        - processed_sheet: pandas dataframe that can be turned into CSV
    '''

    # Define columns to output all data in
    columns = define_columns()

    # Number of models = length of number of tasks
    direct_sheet_data = google_sheet.parse(sheet_names[0])
    cot_sheet_data = google_sheet.parse(sheet_names[1])

    # Iterate through each model column
    for model in all_models:
        
        if task_type == "clf":
            model_name = model
            invalid_name = model + ".3"

        elif task_type == 'ext':
            model_name = model + ".1"
            invalid_name = model + ".2"

        elif task_type == 'gen':
            model_name = model + ".1"
            invalid_name = model + ".3"

        else:
            raise KeyError("Task type not recognized")

        # For each model, iterate through each row in sheet
        for row_idx, score in enumerate(direct_sheet_data[model_name]):
            cot_score = cot_sheet_data[model_name][row_idx]
            tt = direct_sheet_data['Task Type'][row_idx]

            direct_invalid_score = direct_sheet_data[invalid_name][row_idx]
            cot_invalid_score = cot_sheet_data[invalid_name][row_idx]


            # Logic for first row
            if row_idx == 0:
                continue

            # Logic for last row
            if tt == '-':
                break

            direct_score = filter_score(score)
            cot_score = filter_score(cot_score)

            direct_invalid_score = filter_score(direct_invalid_score)
            cot_invalid_score = filter_score(cot_invalid_score)

            invalid_difference = round(cot_invalid_score - direct_invalid_score, 2)

            if direct_invalid_score == 0:
                invalid_relative_difference = 'N/A'
            else:
                invalid_relative_difference = round(invalid_difference / direct_invalid_score, 2)
            

            columns["Model Name"].append(model)
            columns["Model Size"].append(model_mapping[model][0])
            columns["Model Domain"].append(model_mapping[model][1])
            columns["Task Name"].append(direct_sheet_data['Task name'][row_idx])
            columns["Task Type"].append(direct_sheet_data['Task Type'][row_idx])
            columns["Direct Score"].append(direct_score)
            columns["CoT Score"].append(cot_score)
            columns["Direct Invalid Score"].append(direct_invalid_score)
            columns["CoT Invalid Score"].append(cot_invalid_score)
            columns["Difference"].append(round(direct_score - cot_score, 2))
            columns["Invalid Difference"].append(invalid_difference)
            columns["Invalid Relative Difference"].append(invalid_relative_difference)

            if direct_score == 0:
                relative_diff = 0
            else:
                relative_diff = (direct_score - cot_score) / direct_score
            columns["Relative Difference"].append(round(relative_diff, 2))

    return columns


sheet_path = "/Users/kevinxie/Desktop/LLM CoT/LLM-CoT/Reference for Clinical Benchmark and LLM.xlsx"

google_sheet = load_sheet(sheet_path)

clf_data = process_sheet(google_sheet, ['B-CLF', 'B-CLF-CoT'], all_models, 'clf')

ext_data = process_sheet(google_sheet, ['B-EXT', 'B-EXT-CoT'], all_models, 'ext')

gen_data = process_sheet(google_sheet, ['B-GEN', 'B-GEN-CoT'], all_models, 'gen')


# Directly add the calculations



In [3]:
create_csv(clf_data, "CLF-Difference-Sheet-Invalid.csv")

create_csv(ext_data, "EXT-Difference-Sheet-Invalid.csv")

create_csv(gen_data, "Gen-Difference-Sheet-Invalid.csv")

CSV successfully created at CLF-Difference-Sheet-Invalid.csv
CSV successfully created at EXT-Difference-Sheet-Invalid.csv
CSV successfully created at Gen-Difference-Sheet-Invalid.csv
